# 셀레니움을 활용한 동적 크롤링
## 미국 채권 데이터를 python의 selenium 라이브러리를 활용하여 동적으로 크롤링

- 사이트 : https://www.wsj.com/market-data/bonds

- 월 스트리트 저널 홈페이지에는 글로벌 금융시장과 관련된 많은 정보들을 제공하고 있으며, 당연히 미국 채권시장에 대한 정보 또한 실시간으로 제공하고 있다.

- 위의 링크는 월스트리트 저널의 여러 링크 중에서도 미국 국채의 금리와 관련 뉴스, LIBOR 금리, 다른 나라의 주요 국채 금리 등을 제공하고 있는 페이지이다.

- 사이트의 첫 번째 테이블인 미국 국채 유통시장에서 활발히 거래가 되고 있는 대표적인 지표물들의 금리를 크롤링할 예정이다.

- 투자를 위한 판단에 사용할 정확한 채권데이터는 블룸버그를 이용하는 것을 권장한다. 크롤링 학습 및 참고용으로만 사용할 예정이다.

- 이 테이블에 있는 자료는 실시간으로 변하는 자료이기 때문에 동적 크롤링을 사용해야 한다.

- 따라서 파이썬에서 동적으로 데이터를 크롤링 할 수 있는 selenium을 사용한다.

In [5]:
import os

# 크롬 드라이버가 있는 파일 경로를 설정한다.
path = os.getcwd()

# working directory를 변경
os.chdir(path)

In [7]:
# 셀레니움과 특정 웹 브라우저의 드라이버를 사용하여 데이터를 크롤링 할 수 있다.
# 셀레니움 웹 드라이버 선택 (크롬 드라이버 사용)

import pandas as pd
from selenium import webdriver

# 셀레니움 웹드라이브가 웹페이지를 따로 열지 않고도 데이터를 가져올 수 있게 해주는 장치이다.
# 만약 없으면 코드 실행 중 페이지가 자동으로 열린다.
options = webdriver.ChromeOptions()
options.add_argument('headless') # headless: 크롤링 할때 브라우저가 안뜨도록 설정

# 설치한 크롬드라이브의 위치를 webdriver에게 알려줘야 한다. chromedriver.exe가 있는 경로를 설정해준다.
driver = webdriver.Chrome('chromedriver.exe', options=options)

# 크롤링할 해당 페이지의 url를 webdrive에게 알려준다.
URL="https://www.wsj.com/market-data/bonds"
driver.get(URL)

# 현재 크롬드라이버를 사용하여 크롤링한 사이트의 내용에 대한 정보를 가진 셀레니움 객체이다.
# 시간이 걸리는 편임을 알 수 있다.
print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="390f6eb952bf871c2df2f51dc239d352")>


- driver라고 저장된 객체에서 우리가 가져오고자 하는 위치의 내용을 알아야 한다.

- 셀레니움을 사용해 웹스크래핑을 하는 방법에는 여러가지가 있다.

(1) HTML 태그의 class name을 활용하는 방법

- 크롬개발자도구(F12)를 이용하여 해당 위치에 있는 class name을 확인한다. (<tbody class="WSJTables--table__body--3yD9HkB1 "> 라고 나옴)

- 셀레니움의 find_element_by_class_name('클래스 네임') 메서드를 이용하면 해당 테이블에 있는 내용을 가져올 수 있다.

- 테이블 클래스를 가지고 와서 행과 열을 반복문을 사용하여 데이터를 수집할 수 있지만 현재 사이트의 class name은 일정시간마다 바뀐다.

- 쉽고 편한 방법이지만 코드를 일정시간마다 수정해야하므로 이 사이트에서는 사용을 권장하지 않는다.

(2) 데이터의 XPath를 확인하는 방법

- XPath는 프로그래밍 언어의 일종으로 XML Path Language의 약자이며, XML 문서의 구조를 통해 경로(Path)위에 지정한 구문을 사용해 항목을 배치하고 처리하는 방법을 기술한 언어이다. (xml에 대해서는 참고 : https://ko.wikipedia.org/wiki/XML)

- 파이썬과 셀레늄을 사용하기 위해 우리가 원하는 데이터를 수집하기 위해서는 그 데이터가 웹페이지 상의 어디에 위치하는지에 대한 주소값을 알아야 하며 이 경로가 바로 XPath이다.

- 해당 데이터의 XPath는 크롬개발자도구(F12)를 이용하여 우클릭 -> Copy -> Copy XPath를 통해 알아낼 수 있다.

- 주의 해야할 점은 웹페이지의 구조에 따라 XPath의 값이 변경될 수 있다. 이 사이트의 경우 풀화면일때의 XPath와 작은 화면일때의 XPath가 다르다.
    - 작은화면 XPath (subscirbe와 sign in 이 화면에 보임): //*\[@id="root"]/div/div/div/div\[2]/div\[3]/div/div\[3]/div\[1]/div/table/tbody
    - 풀화면 XPath (subscirbe와 sign in 이 화면에 안보임): //*\[@id="root"]/div/div/div/div\[2]/div\[4]/div\[1]/div\[2]/div\[1]/div/table/tbody

- 이처럼 동적인 특정 웹사이트의 구조때문에 원하는 위치의 XPath의 값이 달라질 수 있는지 확인해본다.

In [12]:
# 작은화면의 XPath가 원하는 데이터를 갖고 올 수 있다.

# XPath를 이용한 방법
path = '//*[@id="root"]/div/div/div/div[2]/div[3]/div/div[3]/div[1]/div/table/tbody'

# find_element_by_xpath 메서드를 이용하여 table의 tbody에 대한 정보를 가져온다.
table = driver.find_element_by_xpath(path)

# 테이블은 마찬가지로 셀레니움 객체로 해당 tbody에 대한 정보를 포함하고 있다.
print(table)

# tbody html tag는 tr, td 태그로 둘러싸여 있는데 각 tr, td에 들어있는 text데이터만 뽑아내려면 text메서드를 사용한다.
print(table.text)
print(type(table.text))

# 즉 데이터 전체가 하나의 긴 string으로 되어 있는 데이터다. (매 마지막 줄에 \n이 있는 형태)

<selenium.webdriver.remote.webelement.WebElement (session="390f6eb952bf871c2df2f51dc239d352", element="3ae76423-c3e4-4cfc-9996-ed56b454623f")>
30-Year Bond 1.375 24/32 1.376 -0.005
10-Year Note 0.625 1/32 0.639 -0.008
7-Year Note 0.5 1/32 0.435 -0.014
5-Year Note 0.25 0/32 0.245 -0.003
3-Year Note 0.125 0/32 0.146 0.011
2-Year Note 0.125 0/32 0.129 -0.012
1-Year Bill 0 0/32 0.122 -0.007
6-Month Bill 0 0/32 0.119 -0.011
3-Month Bill 0 0/32 0.102 -0.002
1-Month Bill 0 0/32 0.091 0.005
<class 'str'>


In [11]:
# string data를 dataframe로 변경한다.

rows=table.text.split('\n')
element=[]
for r in rows:
    element.append([i for i in r.split(' ')])
df=pd.DataFrame(element)
print('\n')
headers=['Maturity', 'Type','Coupon','Price Chg', 'Yield', 'Yield Chg']
df.columns=headers
df.iloc[:]=df.iloc[::-1].values

print(df)



  Maturity  Type Coupon Price Chg  Yield Yield Chg
0  1-Month  Bill      0      0/32  0.091     0.005
1  3-Month  Bill      0      0/32  0.104     0.000
2  6-Month  Bill      0      0/32  0.119    -0.011
3   1-Year  Bill      0      0/32  0.124    -0.005
4   2-Year  Note  0.125     22/32  0.129    -0.012
5   3-Year  Note  0.125      0/32  0.141     0.005
6   5-Year  Note   0.25      0/32  0.245    -0.003
7   7-Year  Note    0.5      1/32  0.433    -0.016
8  10-Year  Note  0.625      1/32  0.639    -0.008
9  30-Year  Bond  1.375     25/32  1.368    -0.012


# 뷰티풀숩을 활용한 정적 크롤링
## 미국 채권 데이터를 python의 BeautifulSoup 라이브러리를 활용하여 정적으로 크롤링

- 사이트 : https://www.wsj.com/market-data/bonds 에 있는 각 지표물들의 웹페이지

- 앞에서 동적으로 크롤링한 테이블의 데이터는 현재 지표물들의 금리상황을 알 수 있었지만 구체적인 만기가 나와있지 않았음

- 크롤링 데이터를 활용하여 현재 시점의 국채 금리 커브를 만들기 위해서는 지표물들의 만기 정보가 필수적이므로 해당 정보를 얻을 수 있는 사이트가 필요함

- 따라서 개별 지표물들의 웹페이지(ex. 30년물 : https://www.wsj.com/market-data/quotes/bond/BX/TMUBMUSD30Y?mod=md_bond_overview_quote)로 이동해서 크롤링

- 해당 페이지에서 시장금리(Yield), 채권의 쿠폰금리(Coupon), 해당 채권의 만기(Maturity)를 가져오고자 함

- 위 30년물 페이지 뿐만 아니라 총 10개의 지표물에 대한 정보를 얻기 위해서는 각 지표물에 해당하는 페이지를 돌아다니면서 총 10개의 페이지에 대해 웹 스크래핑을 진행해야 함

## 왜 정적 크롤링인가?

- 동적 크롤링을 수행하는 selenium은 어떤 형태의 웹페이지든지 크롤링할 수 있는 강력한 도구지만 매우 느리다는 단점이 존재

- 따라서 동적 크롤링이 필요없는 상황이라면 selenium보다는 request와 beautifulsoup을 사용하는 것을 고려해볼 만함

- Requests는 기본적으로 파이썬에서 HTTP에 대한 요청을 보내는 모듈이다.

- 다시 말해 이것을 사용해서 우리가 원하는 웹페이지에 대한 정보를 호출할 수 있으며 이를 HTML의 형식으로 받을 수 있다.

- 또 다른 패키지인 BeautifulSoup은 Requests를 통해 받은 HTML을 우리가 이해하기 쉬운 구조로 정리해주는 또 다른 유용한 패키지이다.



In [1]:
import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 총 10개의 지표물에 대한 정보를 얻기 위하여 참고할 페이지에 대한 정보 저장
tenors=['01M', '03M','06M','01Y','02Y','03Y','05Y','07Y','10Y','30Y']

# 만기를 저장할 리스트
maturities=[]

# 시장금리를 저장할 리스트
yields=[]

# 쿠폰금리를 저장할 리스트
coupons=[]


In [2]:
# 특정 웹페이지에서 사람이 아닌 봇같은 사이트 접근 즉, 크롤링 시도가 발견되면 서비스에 지장이 없게 그것들을 어느 정도 막아놓는 장치를 해둔다.
# 네이버도 이런식으로 막아 놓은 장치가 있다. 크롤링하고자 하는 창과 막고자 하는 방패라고 생각하면 된다.
# 이 문제를 해결하는 방법은 크롤링 프로그램이 아닌 사람으로 인식하도록 하는 방법이 있다.
# request를 통해 웹페이지를 요청할 때 headers 옵션을 다음과 같이 해서 넘겨주면 된다.

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

# 이것은 우리가 웹 request 요청을 할때 사용하는 웹브라우저의 user agent에 대한 정보인데 여기 사이트에서 현재 자신이 사용하고 있는 웹브라우저의 user agent를 확인할 수 있다. 
# 참고 : http://www.useragentstring.com/

# 10개 사이트를 반복하여 크롤링한다.
for tenor in tenors:
    # 사이트 구조를 파악한다.
    url = "https://www.wsj.com/market-data/quotes/bond/BX/TMUBMUSD" + tenor + "?mod=md_bond_overview_quote"
    req = requests.get(url,headers=headers)

    # 이렇게 함으로써 해당 페이지의 소스코드 전체가 html이라는 변수에 할당된다.
    html = req.text

    # 이 html에 대한 내용을 beautifulsoup의 html.parser를 이용하여 파싱해준다.
    soup = BeautifulSoup(html,'html.parser')
    
    # 원하는 정보가 있는 위치 찾기. beautifulsoup에선 select 메서드를 사용할 수 있다.

    # soup.select('원하는 정보')  # select('원하는 정보') -->  단 하나만 있더라도, 복수 가능한 형태로 되어있음

    # soup.select('태그명')
    # soup.select('.클래스명')
    # soup.select('상위태그명 > 하위태그명 > 하위태그명')
    # soup.select('상위태그명.클래스명 > 하위태그명.클래스명')    # 바로 아래의(자식) 태그를 선택시에는 > 기호를 사용
    # soup.select('상위태그명.클래스명 하~위태그명')              # 아래의(자손) 태그를 선택시에는   띄어쓰기 사용
    # soup.select('상위태그명 > 바로아래태그명 하~위태그명')     
    # soup.select('.클래스명')
    # soup.select('#아이디명')                  # 태그는 여러개에 사용 가능하나 아이디는 한번만 사용 가능함! ==> 선택하기 좋음
    # soup.select('태그명.클래스명)
    # soup.select('#아이디명 > 태그명.클래스명)
    # soup.select('태그명[속성1=값1]')

    yield_data = soup.select('span[id=quote_val]') # yield data

    # selector를 사용한 방법
    # yield_data = soup.select('body > div > div > section > div > div > div > ul > li > span > span') # yield data
    # print(type(yield_data))
    # print(type(yield_data[0]))
    
    # soup.select('태그명[속성1=값1]')를 사용한 방법
    coupon_and_maturity = soup.select('span[class=data_data]') # coupon rate & maturity date

    # selector를 사용한 방법
    # coupon_and_maturity = soup.select('body > div > div > section > div > div > div > ul > li > div > span') # coupon rate & maturity date
    # print(coupon_and_maturity)

    # 시장금리 값은 데이터의 첫번째 텍스트, 쿠폰금리는 데이터의 세번째 텍스트, 만기는 네번째 텍스트에 저장됨을 확인 
    ytm = yield_data[0].text
    coupon = coupon_and_maturity[2].text
    maturity = coupon_and_maturity[3].text
    
    # 웹스크래핑시 주의해야 할 점은 모든 데이터가 string 즉 문자열임 
    # 시장금리와 쿠폰금리는 숫자여야 하고, 만기는 날짜여야 하기 때문에 데이터 형변환이 필요함
    # 시장금리의 경우 형변환과 마지막 %제거
    ytm = float(ytm[:-1])
    
    # 쿠폰금리의 경우 %제거함. 조건문은 1년 미만 지표물인 경우 쿠폰금리가 존재하지 않고 빈칸으로 나오므로 해당 경우에 값을 0으로 처리
    if coupon!='':
        coupon = float(coupon[:-1])
    else:
        coupon=0.0
    
    # 만기의 경우 월/일/년 형식의 문자열 데이터인데, 앞서 배운 datetime 자료형으로 변환해줌
    maturity_revise = datetime.datetime.strptime(maturity, '%m/%d/%y')

    # 각 리스트에 해당 정보를 추가
    yields.append(ytm)
    coupons.append(coupon)
    maturities.append(maturity_revise)

In [4]:
# 데이터프레임 변환
df = pd.DataFrame([maturities, yields, coupons]).transpose()
headers=['maturity','yield','coupon']
df.columns = headers
df.set_index('maturity',inplace=True)

df

,yield,coupon
maturity,,
2020-09-29,0.096,0
2020-12-03,0.109,0
2021-03-04,0.124,0
2021-08-12,0.129,0
2022-08-31,0.129,0.125
2023-08-15,0.144,0.125
2025-08-31,0.237,0.25
2027-08-31,0.42,0.5
2030-08-15,0.621,0.625


## 어라 그럼 정적크롤링을 하지 왜 첫번째는 동적크롤링 한거?

- 처음 동적 크롤링한 웹사이트의 주소는 https://www.wsj.com/market-data/bonds

- 이 사이트는 Ajax를 활용한 동적으로 제공되는 정보와 정적으로 제공되는 정보가 있음

    - Ajax : 비동기식 자바스크립트 XML(Asynchronous Javascript And XML)의 약자. 
    - 하이퍼텍스트 표기언어(HTML)만으로 어려운 다양한 작업을 웹페이지에서 구현해 이용자가 웹페이지와 자유롭게 상호 작용할 수 있도록 하는 기술
    - 웹페이지를 다시 로딩하지 않고도 메뉴 등 화면상의 객체를 자유롭게 움직이고 다룰 수 있다.

- 그럼 동적으로 제공되는 정보는 뷰티풀 숩을 이용하여 안받아질까? 확인해보자


In [5]:
import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}


url = "https://www.wsj.com/market-data/bonds"
req = requests.get(url,headers=headers)
html = req.text
soup = BeautifulSoup(html,'html.parser')

# tbody 즉 table형태의 정보를 모조리 받아본다. 이걸 가능케 해주는 beautifulsoup 메서드가 find_all
table = soup.find_all('tbody')
print(table)

[<tbody class="WSJTables--table__body--3Y0p0d6H "><tr class="WSJTables--table__row--2VdwxeeP "><td class="WSJTables--table__cell--2u6629rx WSJTables--is-first--f1NiiZJm ">Libor Overnight</td><td class="WSJTables--table__cell--2u6629rx ">0.08250</td><td class="WSJTables--table__cell--2u6629rx ">0.08175</td><td class="WSJTables--table__cell--2u6629rx ">2.18525</td><td class="WSJTables--table__cell--2u6629rx ">0.05075</td></tr><tr class="WSJTables--table__row--2VdwxeeP "><td class="WSJTables--table__cell--2u6629rx WSJTables--is-first--f1NiiZJm ">Libor 1 Week</td><td class="WSJTables--table__cell--2u6629rx ">0.10313</td><td class="WSJTables--table__cell--2u6629rx ">0.11363</td><td class="WSJTables--table__cell--2u6629rx ">2.13263</td><td class="WSJTables--table__cell--2u6629rx ">0.08825</td></tr><tr class="WSJTables--table__row--2VdwxeeP "><td class="WSJTables--table__cell--2u6629rx WSJTables--is-first--f1NiiZJm ">Libor 1 Month</td><td class="WSJTables--table__cell--2u6629rx ">0.15475</td>

In [8]:
# 가장 처음에 있는 테이블에 대한 정보를 가져와서 출력해본다.
tr = table[0].select('tr')
print(tr)



[<tr class="WSJTables--table__row--2VdwxeeP "><td class="WSJTables--table__cell--2u6629rx WSJTables--is-first--f1NiiZJm ">Libor Overnight</td><td class="WSJTables--table__cell--2u6629rx ">0.08250</td><td class="WSJTables--table__cell--2u6629rx ">0.08175</td><td class="WSJTables--table__cell--2u6629rx ">2.18525</td><td class="WSJTables--table__cell--2u6629rx ">0.05075</td></tr>, <tr class="WSJTables--table__row--2VdwxeeP "><td class="WSJTables--table__cell--2u6629rx WSJTables--is-first--f1NiiZJm ">Libor 1 Week</td><td class="WSJTables--table__cell--2u6629rx ">0.10313</td><td class="WSJTables--table__cell--2u6629rx ">0.11363</td><td class="WSJTables--table__cell--2u6629rx ">2.13263</td><td class="WSJTables--table__cell--2u6629rx ">0.08825</td></tr>, <tr class="WSJTables--table__row--2VdwxeeP "><td class="WSJTables--table__cell--2u6629rx WSJTables--is-first--f1NiiZJm ">Libor 1 Month</td><td class="WSJTables--table__cell--2u6629rx ">0.15475</td><td class="WSJTables--table__cell--2u6629rx "

In [ ]:
# 바로 Libor금리가 나오는 것으로 보아 동적데이터는 뷰티풀숲으로 안가져와 지고있는 상황이다.
# 동적데이터는 selenium, 정적데이터는 beautifulsoup으로 가져와야 하는 이유다.


## 정적 크롤링(beautiful soup), 동적 크롤링(selenium) 모두 사용

### 지표물, 비지표물에 대한 채권데이터 가져오기(월스트리트 저널)

- 사이트 : https://www.wsj.com/market-data/bonds/treasuries

- 정적크롤링으로 미국 국채 지표물에 대해 월스트리트저널 페이지를 돌며 필요한 데이터들을 수집하였다.

- 지표물은 어떤 특정 만기의 채권을 대표하는 가장 유동성이 좋은 대표 종목이기 때문에 시장에서 거래가 많이 되며, 또한 이 지표물을 통해 수익률 곡선을 만든다

- 그렇기 때문에 현재 채권시장에서 어떤 종목이 지표물인지를 알고 있는 것은 매우 중요하다.

- 이번에는 지표물 이외에도 현재 유통시장에 나와있는 모든 미국 국채들에 대한 데이터를 웹스크래핑 하는 방법을 알아보자

- 채권시장에서는 지표물 이외에도 현재 유통시장에 나와있고 아직 잔존만기가 존재하고 있는 비지표물이 있다.

- 이러한 비지표물들은 발행시점 당시에는 지표물이었으나 만기가 줄어들고 새로운 종목이 발행되면서 비지표물이 된다.

- 비지표물 데이터를 크롤링하는데 Treasury Notes & Bond와 Treasury Bills가 클릭으로 구분되어 있다.

- 클릭을 구현하기 위해 동적 크롤링을 수행하는 selenium을 사용한다.

### 간단한 사전 지식

- 해당 사이트의 데이터는 Tullett Prebon 사에서 제공한다. 이 회사는 채권 데이터를 중개해주는 회사이며 블룸버그의 채권데이터도 이 회사가 제공한다.

- Treasury Bills는 만기 1년 이하, Treasury Notes는 만기 2~5년의 종목, Treasury Bonds는 만기 10~30년의 종목들을 의미한다. 여기서 만기는 발행 시점 당시의 잔존만기를 의미한다.

- 해당 사이트의 데이터는 장외 채권시장에서 미국 동부시간 기준 오후 3시에 시장에서 호가 된 가격을 보여준다.

- 만기 이전에 상환할 수 있는 수의상환사채(callable bond)의 경우에 시장금리는 액면가 이상인 발행물에 대해서는 가장 이른 콜 일자로 계산되고, 액면가 미만인 발행물에 대해서는 만기일로 계산됩니다. 아래는 채권에 옵션의 성격이 들어가 있는 콜 가능 채권(callable bond)와 풋 가능 채권(puttable bond)의 예시입니다.

    - 수의상환사채(callable bond) : 발행자가 만기일 전에 상환을 행사 할 수 있는 사채.  채무자 : 콜 매입 포지션, 채권자 : 콜 매도 포지션

    - 조기상환청구권(상환청구사채) :  채권자가 만기일 전에 채권을 팔 수 있는 권리. 채무자 : 풋 매도 포지션, 채권자 : 픗 매입 포지션(매도청구권)

In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument('headless') # headless: 브라우저가 안뜸

driver = webdriver.Chrome('chromedriver.exe', options=options)
URL="https://www.wsj.com/market-data/bonds/treasuries"
driver.get(URL)

# 기본적으로 Treasury Notes & Bond(만기 2~10년 이하, 10~30년 이하)의 비지표물이 나와있는 페이지가 보임
# 해당 웹페이지에 대한 소스코드를 수집하여 뷰티풀숩을 사용해 해당 페이지의 html에 대한 정보를 변수에 저장한다.
source = driver.page_source
tbonds_code = BeautifulSoup(source,'html.parser')

# 버튼을 클릭해서 만기 1년 이하의 Treasury Bills 비지표물에 대한 사이트를 표시해야한다.
# 크롬개발자도구를 이용해서 찾은 xpath을 확인하여 버튼 위치를 찾고 클릭한다.
# 셀레니움이 이 작업을 사람을 대신해서 한다
button_xpath = '//*[@id="root"]/div/div/div/div[2]/div/div/div[3]/ul/li[2]/button'
button = driver.find_element_by_xpath(button_xpath)
button.click()

# 해당 웹페이지에 대한 소스코드를 수집하여 뷰티풀숩을 사용해 해당 페이지의 html에 대한 정보를 변수에 저장한다.
source = driver.page_source
tbills_code = BeautifulSoup(source,'html.parser')


In [3]:
#date 가져오기 
date = tbonds_code.select('div > div > div > div > div > div > div > div > h3 > span')
data = date[1].text
print(data)
print('\n')

#tbond table 가져오기
tbond_rows = tbonds_code.select('div > div > div > div > div > div > div > div > table > tbody > tr')
print(tbond_rows)
print('\n')

#tbill table 가져오기
tbill_rows = tbills_code.select('div > div > div > div > div > div > div > div > table > tbody > tr')
print(tbill_rows)
print('\n')


Tuesday, October 27, 2020


[<tr class="WSJTables--table__row--2VdwxeeP "><td class="WSJTables--table__cell--2u6629rx WSJTables--is-first--f1NiiZJm WSJTheme--table__cell--3njwWeaF ">10/31/2020</td><td class="WSJTables--table__cell--2u6629rx WSJTheme--table__cell--3njwWeaF ">1.375</td><td class="WSJTables--table__cell--2u6629rx WSJTheme--table__cell--3njwWeaF ">100.0100</td><td class="WSJTables--table__cell--2u6629rx WSJTheme--table__cell--3njwWeaF ">100.0140</td><td class="WSJTables--table__cell--2u6629rx WSJTables--u-positive--3zGr9GY4 WSJBase--u-positive--2jiH4U3E WSJTheme--table__cell--3njwWeaF ">0.0080</td><td class="WSJTables--table__cell--2u6629rx WSJTheme--table__cell--3njwWeaF ">-2.8958</td></tr>, <tr class="WSJTables--table__row--2VdwxeeP "><td class="WSJTables--table__cell--2u6629rx WSJTables--is-first--f1NiiZJm WSJTheme--table__cell--3njwWeaF ">10/31/2020</td><td class="WSJTables--table__cell--2u6629rx WSJTheme--table__cell--3njwWeaF ">1.750</td><td class="WSJTables--table__

In [4]:
# 즉 원하는 위치에 해당하는 모든 tr에 대한 정보가 tbond_rows와 tbill_row에 저장되어 있다.

# tr:행 td:열 이므로 반복문을 통해 해당 tr에 존재하는 모든 td(열)을 찾고 td 태그안에 존재하는 텍스트를 리스트에 저장한다.
# tbond에 저장

tbond_content = []
tbond_contents = []

# tbond의 모든 tr태그가 존재하는 리스트에서 반복문으로 각 tr태그 선택
for tbond_row in tbond_rows:

    # 해당 tr 태그에서 존재하는 모든 td를 찾음. 각원소가 td tag를 가진 리스트로 저장됨 
    tds = tbond_row.find_all("td")

    # 마찬가지로 모든 td태그가 존재하는 리스트에서 반복문으로 각 td태그 선택
    for td in tds:

        # td 태그안에 존재하는 텍스트를 리스트에 추가
        tbond_content.append(td.text)
    # td 태그안 정보가 있는 리스트를 추가. tbond_contents 리스트의 각 원소는 해당 tr(행)의 정보들이 있는 리스트임
    tbond_contents.append(tbond_content)
    
    # td 태그 정보는 초기화
    tbond_content=[]

# 데이터프레임으로 변환
tbond_df=pd.DataFrame(tbond_contents)
tbond_headers=['Maturity', 'Coupon','Bid','Ask','Chg', 'Asked Yield']
tbond_df.columns=tbond_headers
tbond_df.set_index('Maturity',inplace=True)

tbond_df

,Coupon,Bid,Ask,Chg,Asked Yield
Maturity,,,,,
10/31/2020,1.375,100.0100,100.0140,0.0080,-2.8958
10/31/2020,1.750,100.0100,100.0140,0.0080,-2.5239
10/31/2020,2.875,100.0120,100.0160,0.0080,-2.1142
11/15/2020,1.750,100.0240,100.0300,unch.,-0.165
11/15/2020,2.625,100.0360,100.0420,unch.,-0.089
...,...,...,...,...,...
8/15/2049,2.250,116.0940,116.1140,0.8780,1.544
11/15/2049,2.375,119.1260,119.1460,0.8760,1.541
2/15/2050,2.000,110.1420,110.1620,0.8680,1.552


In [13]:
# 똑같은 과정을 tbill에 대해서도 수행

tbill_content = []
tbill_contents = []

for tbill_row in tbill_rows:
    tds = tbill_row.find_all("td")
    for td in tds:
        tbill_content.append(td.text)
    tbill_contents.append(tbill_content)
    tbill_content=[]

tbill_df=pd.DataFrame(tbill_contents)
tbill_headers=['Maturity','Bid','Ask','Chg', 'Asked Yield']
tbill_df.columns=tbill_headers
tbill_df.set_index('Maturity',inplace=True)

tbill_df

,Bid,Ask,Chg,Asked Yield
Maturity,,,,
9/8/2020,0.090,0.080,-0.010,0.081
9/10/2020,0.090,0.080,-0.005,0.081
9/15/2020,0.095,0.085,+0.008,0.086
9/17/2020,0.098,0.088,+0.008,0.089
9/22/2020,0.100,0.090,+0.008,0.091
9/24/2020,0.093,0.083,unch.,0.084
9/29/2020,0.095,0.085,+0.003,0.086
10/1/2020,0.093,0.083,unch.,0.084
10/6/2020,0.093,0.083,-0.008,0.084
